## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,63.99,80,43,12.77,light rain
1,1,Panabá,MX,21.2833,-88.2667,79.56,74,55,7.65,broken clouds
2,2,Mayo,CA,63.5927,-135.8998,39.70,71,39,1.97,scattered clouds
3,3,North Battleford,CA,52.7834,-108.2847,49.41,55,29,8.48,light rain
4,4,Ushuaia,AR,-54.8000,-68.3000,40.66,75,75,23.02,light intensity drizzle


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Please enter the minimum temperature (F) for your search: '))
max_temp = float(input('Please enter the maximum temperature (F) for your search: '))

Please enter the minimum temperature (F) for your search: 73
Please enter the maximum temperature (F) for your search: 85


In [31]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df['Max Temp (F)'] >= min_temp) & \
    (city_data_df['Max Temp (F)'] <= max_temp)
]
preferred_cities_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Current Description
1,1,Panabá,MX,21.2833,-88.2667,79.56,74,55,7.65,broken clouds
6,6,Buchanan,LR,5.8808,-10.0467,76.30,87,100,5.99,light rain
14,14,Erbil,IQ,36.1926,44.0106,75.92,26,0,1.77,clear sky
15,15,Alofi,NU,-19.0595,-169.9187,77.63,72,3,5.28,light rain
23,23,Kapaa,US,22.0752,-159.3190,80.58,71,83,1.99,broken clouds
...,...,...,...,...,...,...,...,...,...,...
671,671,Morant Bay,JM,17.8815,-76.4093,76.62,81,51,6.49,broken clouds
672,672,Digha,IN,21.6833,87.5500,76.84,91,100,4.92,moderate rain
677,677,Kashgar,CN,39.4547,75.9797,78.08,16,0,2.24,clear sky
680,680,Saint-Georges,GF,3.9000,-51.8000,73.13,95,84,2.06,broken clouds


In [32]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())

Unnamed: 0             249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp (F)           249
Humidity (%)           249
Cloudiness (%)         249
Wind Speed (mph)       249
Current Description    249
dtype: int64


In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

print(clean_df.count())

Unnamed: 0             249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp (F)           249
Humidity (%)           249
Cloudiness (%)         249
Wind Speed (mph)       249
Current Description    249
dtype: int64


In [74]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp (F)", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ''
hotel_df.head(10)

,City,Country,Max Temp (F),Current Description,Lat,Lng,Hotel Name
1,Panabá,MX,79.56,broken clouds,21.2833,-88.2667,
6,Buchanan,LR,76.30,light rain,5.8808,-10.0467,
14,Erbil,IQ,75.92,clear sky,36.1926,44.0106,
15,Alofi,NU,77.63,light rain,-19.0595,-169.9187,
23,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,
27,Sakakah,SA,77.68,clear sky,29.9697,40.2064,
29,Rikitea,PF,74.01,clear sky,-23.1203,-134.9692,
30,Bengkulu,ID,80.98,scattered clouds,-3.8004,102.2655,
31,Bubaque,GW,81.46,overcast clouds,11.2833,-15.8333,
35,Gurupi,BR,82.18,clear sky,-11.7292,-49.0686,


In [75]:
hotel_df.dropna()
hotel_df.head(10)
hotel_df.count()

City                   249
Country                249
Max Temp (F)           249
Current Description    249
Lat                    249
Lng                    249
Hotel Name             249
dtype: int64

In [76]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # location = f'{lat},{lng}'
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # url for nearby search
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json' # ?parameters'
    # base_url = 'https://maps.googleapis.com/maps/api/directions/json' # ?origin=Disneyland&destination=Universal+Studios+Hollywood&key=YOUR_API_KEY'

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url, params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_name = response['results'][0]['name']
    except:
        continue
    
    hotel_df.loc[i, 'Hotel Name'] = hotel_name
        

In [77]:
print(len(hotel_df))

249


In [83]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df

for i, row in hotel_clean_df.iterrows():
    if row['Hotel Name'] == '':
        clean_hotel_df.drop([i], inplace=True)

print(len(hotel_clean_df))
clean_hotel_df.head(10)

239


,City,Country,Max Temp (F),Current Description,Lat,Lng,Hotel Name
1,Panabá,MX,79.56,broken clouds,21.2833,-88.2667,Posada rosario
6,Buchanan,LR,76.30,light rain,5.8808,-10.0467,Kamaneahn Hotel Inc
14,Erbil,IQ,75.92,clear sky,36.1926,44.0106,Erbil International Hotel
15,Alofi,NU,77.63,light rain,-19.0595,-169.9187,Taloa Heights
23,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
27,Sakakah,SA,77.68,clear sky,29.9697,40.2064,Fakhamat Al Orjoana Apartment
29,Rikitea,PF,74.01,clear sky,-23.1203,-134.9692,People ThankYou
30,Bengkulu,ID,80.98,scattered clouds,-3.8004,102.2655,Grage Hotel Bengkulu
31,Bubaque,GW,81.46,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
35,Gurupi,BR,82.18,clear sky,-11.7292,-49.0686,Via Norte Hotel


In [84]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [89]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp (F)} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [91]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,
                                  info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))